In [1]:
from xclib.data import data_utils
import numpy as np
import time


In [2]:
#Change this later. Make the path as command line argument
X_train = data_utils.read_sparse_file('ass3_parta_data/train_x.txt', force_header=True)
train_class = np.genfromtxt('ass3_parta_data/train_y.txt').reshape(-1, 1)

X_valid = data_utils.read_sparse_file('ass3_parta_data/valid_x.txt', force_header=True)
valid_class = np.genfromtxt('ass3_parta_data/valid_y.txt').reshape(-1, 1)

X_test = data_utils.read_sparse_file('ass3_parta_data/test_x.txt', force_header=True)
test_class = np.genfromtxt('ass3_parta_data/test_y.txt').reshape(-1, 1)

/home/ayushi/.local/lib/python3.6/site-packages/xclib-0.96-py3.6-linux-x86_64.egg/xclib/data/data_utils.py:173: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [3]:
f = open('ass3_parta_data/train_x.txt') 
m,n = f.readlines()[0].rstrip("\n").split(" ")
m,n = int(m),int(n)
f.close()

In [4]:
classes = np.unique(train_class)
pos_class_ind = np.where(train_class == 1)
neg_class_ind = np.where(train_class == 0)

In [5]:
attributes = np.array(range(X_train.A.shape[1]))

### Entropy H(Y) is calculated assuming Y only has two classes - 0 and 1

###### So we use log_2 and use pos and neg_num

In [6]:
def calc_entropyY(train_data, train_class):
    pos_num = train_data[np.where(train_class == 1)[0]].shape[0]
    neg_num = train_data[np.where(train_class == 0)[0]].shape[0]
    
    if (pos_num == 0) or (neg_num == 0):
        entr = 0 #in the limit
    else:
        entr = (pos_num/(pos_num+neg_num))*np.log2((pos_num+neg_num)/pos_num) + (neg_num/(pos_num+neg_num))*np.log2((pos_num+neg_num)/neg_num)
        
    return entr

In [7]:
#Entropy of Raw data
start = time.time()
print(calc_entropyY(X_train.A, train_class))
print("time = ", time.time()-start)

0.997775157550985
time =  0.1013479232788086


#### Reduction in Error is given by H(Y|X) by using value to split at the node
H(Y|X) 

       = H(Y|X=x)P(X=x)  -- find the two splits 

       = H(Y=y|X=x)P(X=x) -- find the entropy of both the splits. 

       = calc_entropy(split1, train_class_split1).P(X=x)
    

In [8]:
def calc_red_error(train_data, split, split_class, train_class):
    #print("In reduction of Error Module")
    #print(train_data.shape[0])
    error = 0
    for i in range(len(split)):
    #    print("Split shape = ", split[i].shape)
        error += (split[i].shape[0]/train_data.shape[0])*calc_entropyY(split[i], split_class[i])
    #print("Reduction in error = ", error)
    return error


In [111]:
def find_best_attr(train_data, train_class):
    #max_mutual_info = 0;   This doesnt work for the cases when mutual info becomes negative. 
    max_mutual_info = float("-inf");
    median = (np.median(train_data, axis=0))        
    #print(median)
    for i in range(len(attributes)):
        split = []
        split_class = []
        split.append(train_data[np.where(train_data[:,i] <= median[i])])
        split.append(train_data[np.where(train_data[:,i] > median[i])])
        split_class.append(train_class[np.where(train_data[:,i] <= median[i])])
        split_class.append(train_class[np.where(train_data[:,i] > median[i])])
        mutual_info = calc_entropyY(train_data, train_class) - calc_red_error(train_data, split, split_class, train_class)
        #print(mutual_info)
        if max_mutual_info < mutual_info:
            max_mutual_info = mutual_info
            best_attr = attributes[i]
            thres = median[i]
    return max_mutual_info, best_attr, thres

In [105]:
start = time.time()
print(find_best_attr(test[0], test[2]))
print(time.time()-start)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4

4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.99602563376

Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4677, 482)
Split shape =  (0, 482)
Reduction in error =  0.9960256337673982
0.0
4677
Split shape =  (4

### Every Node  in the tree will have the following contents:

1. Left Subtree
2. Right Subtree
3. Attribute Name on which this node will split
4. Threshold of the attribute for left and right subtree (Median) - int value
5. Entropy
6. Whether this node is leaf or not - True/False
7. A final value of Y=0 or Y=1 in case of leaf

In [67]:
class newNode(object):
    def __init__(self, attr, Th, entr):
        self.left = None
        self.right = None
        self.attribute = attr
        self.threshold = Th
        self.entropy = entr
        self.leaf = False
        self.final = None

In [115]:
def growTree(train_data, train_class, num_nodes):
    pos_num = train_data[np.where(train_class == 1)[0]].shape[0]
    neg_num = train_data[np.where(train_class == 0)[0]].shape[0]
    print("The number of nodes present in the graph = ", num_nodes)
    print(neg_num, pos_num)
    
    if(pos_num == 0 or neg_num == 0):
        num_nodes+=1
        leaf = newNode(None, None, None)
        leaf.leaf = True
        if(pos_num != 0):
            leaf.final = 1
        elif(neg_num != 0):
            leaf.final = 0
        return leaf
        
    else:
        num_nodes+=1
        mutual_info, best_attr, thresh = find_best_attr(train_data, train_class)
        print("Best Attribute with Threshold = ", best_attr, thresh)

        addNode = newNode(best_attr, thresh, calc_entropyY(train_data, train_class))
        #print("node Entropy=", addNode.entropy)
        #print("Node data size = ", train_data.shape)
        left_split = train_data[np.where(train_data[:,best_attr] <= thresh)]
        right_split = train_data[np.where(train_data[:,best_attr] > thresh)]
        left_split_class = train_class[np.where(train_data[:,best_attr] <= thresh)]
        right_split_class = train_class[np.where(train_data[:,best_attr] > thresh)]
        #if (left_split.shape)
        if (left_split.shape[0] == 0):
            leaf = newNode(None, None, None)
            leaf.leaf = True
            pos = left_split[np.where(left_split_class == 1)[0]].shape[0]
            neg = left_split[np.where(left_split_class == 0)[0]].shape[0]
            if (pos>neg):
                leaf.final = 1
            else:
                leaf.final = 0
            return leaf
        elif (right_split.shape[0] == 0):
            leaf = newNode(None, None, None)
            leaf.leaf = True
            pos = right_split[np.where(right_split_class == 1)[0]].shape[0]
            neg = right_split[np.where(right_split_class == 0)[0]].shape[0]
            if (pos>neg):
                leaf.final = 1
            else:
                leaf.final = 0
            return leaf
        else:
            print("Building left tree on =", left_split.shape[0])
            addNode.left = growTree(left_split, left_split_class, num_nodes)
            print("Building right tree in = ", right_split.shape[0])
            addNode.right = growTree(right_split, right_split_class, num_nodes)
            return addNode

In [116]:
start = time.time()
#test=[]
root = growTree(X_train.A, train_class, 0)
print("Time Taken to grow tree = ", time.time() - start)

The number of nodes present in the graph =  0
30560 34153
Best Attribute with Threshold =  217 0.0
Building left tree on = 47691
The number of nodes present in the graph =  1
17868 29823
Best Attribute with Threshold =  252 0.0
Building left tree on = 43010
The number of nodes present in the graph =  2
17724 25286
Best Attribute with Threshold =  57 0.0
Building left tree on = 39800
The number of nodes present in the graph =  3
17359 22441
Best Attribute with Threshold =  401 0.0
Building left tree on = 37755
The number of nodes present in the graph =  4
17202 20553
Best Attribute with Threshold =  61 1.0
Building left tree on = 19479
The number of nodes present in the graph =  5
7625 11854
Best Attribute with Threshold =  83 0.0
Building left tree on = 15539
The number of nodes present in the graph =  6
6882 8657
Best Attribute with Threshold =  206 0.0
Building left tree on = 14122
The number of nodes present in the graph =  7
5916 8206
Best Attribute with Threshold =  239 0.0
Buildi

Best Attribute with Threshold =  34 0.5
Building left tree on = 2
The number of nodes present in the graph =  28
2 0
Building right tree in =  2
The number of nodes present in the graph =  28
1 1
Best Attribute with Threshold =  141 43.0
Building left tree on = 1
The number of nodes present in the graph =  29
1 0
Building right tree in =  1
The number of nodes present in the graph =  29
0 1
Building right tree in =  1
The number of nodes present in the graph =  24
1 0
Building right tree in =  59
The number of nodes present in the graph =  23
0 59
Building right tree in =  92
The number of nodes present in the graph =  22
0 92
Building right tree in =  1
The number of nodes present in the graph =  21
1 0
Building right tree in =  7
The number of nodes present in the graph =  20
4 3
Best Attribute with Threshold =  151 4.0
Building left tree on = 5
The number of nodes present in the graph =  21
2 3
Best Attribute with Threshold =  84 0.0
Building left tree on = 3
The number of nodes pre

Best Attribute with Threshold =  146 0.0
Building left tree on = 50
The number of nodes present in the graph =  17
44 6
Best Attribute with Threshold =  121 0.0
Building left tree on = 34
The number of nodes present in the graph =  18
28 6
Best Attribute with Threshold =  240 0.0
Building left tree on = 33
The number of nodes present in the graph =  19
28 5
Best Attribute with Threshold =  84 8.0
Building left tree on = 30
The number of nodes present in the graph =  20
25 5
Best Attribute with Threshold =  0 0.0
Building left tree on = 28
The number of nodes present in the graph =  21
23 5
Best Attribute with Threshold =  141 51.0
Building left tree on = 18
The number of nodes present in the graph =  22
14 4
Best Attribute with Threshold =  27 8.0
Building left tree on = 10
The number of nodes present in the graph =  23
9 1
Best Attribute with Threshold =  34 0.0
Building left tree on = 8
The number of nodes present in the graph =  24
8 0
Building right tree in =  2
The number of nodes

Best Attribute with Threshold =  87 0.0
Building left tree on = 157
The number of nodes present in the graph =  20
10 147
Best Attribute with Threshold =  80 0.0
Building left tree on = 156
The number of nodes present in the graph =  21
9 147
Best Attribute with Threshold =  84 0.0
Building left tree on = 104
The number of nodes present in the graph =  22
4 100
Best Attribute with Threshold =  151 1.0
Building left tree on = 84
The number of nodes present in the graph =  23
4 80
Best Attribute with Threshold =  107 25947.0
Building left tree on = 42
The number of nodes present in the graph =  24
1 41
Best Attribute with Threshold =  107 9242.5
Building left tree on = 21
The number of nodes present in the graph =  25
1 20
Best Attribute with Threshold =  100 0.0
Building left tree on = 11
The number of nodes present in the graph =  26
0 11
Building right tree in =  10
The number of nodes present in the graph =  26
1 9
Best Attribute with Threshold =  107 525.5
Building left tree on = 5


Best Attribute with Threshold =  34 0.0
Building left tree on = 48
The number of nodes present in the graph =  17
6 42
Best Attribute with Threshold =  203 0.0
Building left tree on = 27
The number of nodes present in the graph =  18
6 21
Best Attribute with Threshold =  20 0.0
Building left tree on = 20
The number of nodes present in the graph =  19
6 14
Best Attribute with Threshold =  107 72.0
Building left tree on = 11
The number of nodes present in the graph =  20
6 5
Best Attribute with Threshold =  84 5.0
Building left tree on = 9
The number of nodes present in the graph =  21
4 5
Best Attribute with Threshold =  114 0.0
Building left tree on = 8
The number of nodes present in the graph =  22
3 5
Best Attribute with Threshold =  151 32.0
Building left tree on = 5
The number of nodes present in the graph =  23
1 4
Best Attribute with Threshold =  100 4.0
Building left tree on = 4
The number of nodes present in the graph =  24
0 4
Building right tree in =  1
The number of nodes pr

...

Best Attribute with Threshold =  107 156.0
Building left tree on = 15
The number of nodes present in the graph =  8
1 14
Best Attribute with Threshold =  108 0.0
Building left tree on = 12
The number of nodes present in the graph =  9
0 12
Building right tree in =  3
The number of nodes present in the graph =  9
1 2
Best Attribute with Threshold =  5 12.0
Building left tree on = 2
The number of nodes present in the graph =  10
0 2
Building right tree in =  1
The number of nodes present in the graph =  10
1 0
Building right tree in =  15
The number of nodes present in the graph =  8
10 5
Best Attribute with Threshold =  230 11.0
Building left tree on = 8
The number of nodes present in the graph =  9
3 5
Best Attribute with Threshold =  225 5.0
Building left tree on = 4
The number of nodes present in the graph =  10
3 1
Best Attribute with Threshold =  8 0.5
Building left tree on = 2
The number of nodes present in the graph =  11
2 0
Building right tree in =  2
The number of nodes presen

Best Attribute with Threshold =  115 42.0
Building left tree on = 6
The number of nodes present in the graph =  19
0 6
Building right tree in =  5
The number of nodes present in the graph =  19
3 2
Best Attribute with Threshold =  203 74.0
Building left tree on = 3
The number of nodes present in the graph =  20
1 2
Best Attribute with Threshold =  19 138.0
Building left tree on = 2
The number of nodes present in the graph =  21
1 1
Best Attribute with Threshold =  19 126.5
Building left tree on = 1
The number of nodes present in the graph =  22
1 0
Building right tree in =  1
The number of nodes present in the graph =  22
0 1
Building right tree in =  1
The number of nodes present in the graph =  21
0 1
Building right tree in =  2
The number of nodes present in the graph =  20
2 0
Building right tree in =  2
The number of nodes present in the graph =  18
2 0
Building right tree in =  3
The number of nodes present in the graph =  16
0 3
Building right tree in =  6
The number of nodes pr

Best Attribute with Threshold =  165 0.0
Building left tree on = 23
The number of nodes present in the graph =  12
9 14
Best Attribute with Threshold =  151 138.0
Building left tree on = 13
The number of nodes present in the graph =  13
2 11
Best Attribute with Threshold =  100 8.0
Building left tree on = 9
The number of nodes present in the graph =  14
0 9
Building right tree in =  4
The number of nodes present in the graph =  14
2 2
Best Attribute with Threshold =  15 29.5
Building left tree on = 2
The number of nodes present in the graph =  15
2 0
Building right tree in =  2
The number of nodes present in the graph =  15
0 2
Building right tree in =  10
The number of nodes present in the graph =  13
7 3
Best Attribute with Threshold =  63 5.0
Building left tree on = 5
The number of nodes present in the graph =  14
5 0
Building right tree in =  5
The number of nodes present in the graph =  14
2 3
Best Attribute with Threshold =  20 1.0
Building left tree on = 3
The number of nodes pr

Best Attribute with Threshold =  119 1.0
Building left tree on = 108
The number of nodes present in the graph =  7
6 102
Best Attribute with Threshold =  88 25.0
Building left tree on = 81
The number of nodes present in the graph =  8
6 75
Best Attribute with Threshold =  24 699.0
Building left tree on = 43
The number of nodes present in the graph =  9
5 38
Best Attribute with Threshold =  173 562.0
Building left tree on = 23
The number of nodes present in the graph =  10
1 22
Best Attribute with Threshold =  79 8.0
Building left tree on = 18
The number of nodes present in the graph =  11
0 18
Building right tree in =  5
The number of nodes present in the graph =  11
1 4
Best Attribute with Threshold =  8 0.0
Building left tree on = 3
The number of nodes present in the graph =  12
0 3
Building right tree in =  2
The number of nodes present in the graph =  12
1 1
Best Attribute with Threshold =  15 349.0
Building left tree on = 1
The number of nodes present in the graph =  13
0 1
Buildi

Best Attribute with Threshold =  232 7.0
Building left tree on = 82
The number of nodes present in the graph =  11
5 77
Best Attribute with Threshold =  107 120.0
Building left tree on = 45
The number of nodes present in the graph =  12
1 44
Best Attribute with Threshold =  115 1.0
Building left tree on = 37
The number of nodes present in the graph =  13
0 37
Building right tree in =  8
The number of nodes present in the graph =  13
1 7
Best Attribute with Threshold =  186 0.0
Building left tree on = 6
The number of nodes present in the graph =  14
0 6
Building right tree in =  2
The number of nodes present in the graph =  14
1 1
Best Attribute with Threshold =  27 48.0
Building left tree on = 1
The number of nodes present in the graph =  15
1 0
Building right tree in =  1
The number of nodes present in the graph =  15
0 1
Building right tree in =  37
The number of nodes present in the graph =  12
4 33
Best Attribute with Threshold =  135 1.0
Building left tree on = 23
The number of no

In [ ]:
with open('mypickle.pickle') as f:
    root = pickle.load(f)

In [118]:
def predict(tree, data):
    if tree.leaf:
        return tree.final
    if data[tree.attribute] <= tree.threshold:
        return predict(tree.left, data)
    elif data[tree.attribute] > tree.threshold:
        return predict(tree.right, data)


In [137]:
test_pred = []
acc = 0
for i in range(len(X_test.A)):
    test_pred.append(predict(root, X_test.A[i]))
    if (test_pred[i] == test_class[i]):
        acc+=1
        

print(acc/len(test_class))


15461


In [ ]:
train_pred = []
acc = 0
for i in range(len(X_train.A)):
    train_pred.append(predict(root, X_train.A[i]))
    if (train_pred[i] == train_class[i]):
        acc+=1

print("The train Accuracy is = ", acc/len(train_class))


In [ ]:
def calc_accuracy(tree, data, classes):
    acc = 0
    for i in range(len(data)):
        pred = predict(tree, data[i])
        if (pred == classes[i]):
            acc+=1
    return acc/len(classes)

In [ ]:
#train_acc = calc_accuracy(root, X_train.A, train_class)
val_acc = calc_accuracy(root, X_val.A, val_class)
#test_acc = calc_accuracy(root, X_test.A, test_class)

Test Accuracy = 71.67%
Train Accuracy = 
Validation Accuracy = 

In [5]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, train_class)

In [6]:
tree.plot_tree(clf) 

[Text(319.05051113061876, 366.52, 'X[217] <= 1.5\ngini = 0.498\nsamples = 64713\nvalue = [30560, 34153]'),
 Text(257.36407037011065, 360.35999999999996, 'X[363] <= 31.5\ngini = 0.468\nsamples = 47727\nvalue = [17872, 29855]'),
 Text(225.01246732417226, 354.2, 'X[57] <= 0.5\ngini = 0.489\nsamples = 41489\nvalue = [17697, 23792]'),
 Text(174.68336964363738, 348.03999999999996, 'X[401] <= 0.5\ngini = 0.496\nsamples = 38291\nvalue = [17335, 20956]'),
 Text(88.80787074189956, 341.88, 'X[151] <= 8.5\ngini = 0.499\nsamples = 36235\nvalue = [17178, 19057]'),
 Text(34.19124282323241, 335.71999999999997, 'X[107] <= 13.5\ngini = 0.474\nsamples = 14650\nvalue = [5639, 9011]'),
 Text(21.498475631824196, 329.55999999999995, 'X[106] <= 0.5\ngini = 0.493\nsamples = 10045\nvalue = [4437, 5608]'),
 Text(19.75143691552121, 323.4, 'X[21] <= 0.5\ngini = 0.491\nsamples = 9804\nvalue = [4240, 5564]'),
 Text(17.7520326703668, 317.23999999999995, 'X[353] <= 0.5\ngini = 0.489\nsamples = 9692\nvalue = [4130, 556